# File 09: Prediction Sentiment Analysis on Main Dataset

### Input Files:
- model-sa/model-gpu.yaml
- model-sa/model-weights-gpu.h5
- model-sa/tokenizer.pickle
- db/04-main-data.csv
- db/08-user-rating.csv

### Output Files:
- db/09-main-prediction.csv

### Steps:
1. loading required libraries
1. loading model with weights
1. loading the tokenizer
1. loading main dataset
1. dropping unnecessary rows
1. prepping input for prediction
1. predicting sentiment
1. fetching user rating
1. saving output

In [1]:
# loading required libraries
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from keras.models import model_from_yaml
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
from sklearn.metrics import confusion_matrix, accuracy_score

Using TensorFlow backend.


In [2]:
# loading model with weights
def load_model(model, weight) :
    with open(model, 'r') as file:
        yaml_model = file.read()

    model = tf.keras.models.model_from_yaml(yaml_model)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.load_weights(weight)
    return model

model = load_model('sentiment-analysis-model/model-gpu.yaml', 'sentiment-analysis-model/weights-gpu.h5')
model.summary()

C:\Users\Artemis\miniconda3\envs\project\lib\site-packages\tensorflow_core\python\keras\saving\model_config.py:76: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yaml_string)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 48, 128)           512000    
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 48, 128)           0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 48, 196)           255584    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 48, 196)           0         
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 196)               308896    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 1,076,874
Trainable params: 1,076,874
Non-trainable params: 0
____________________________________________

In [3]:
# loading the tokenizer
with open('sentiment-analysis-model/tokenizer.pickle', 'rb') as file:
    tokenizer = pickle.load(file)

In [70]:
# loading main dataset
df = pd.read_csv("db/04-main-data.csv")
rating = pd.read_csv("db/08-user-rating.csv")

In [71]:
# dropping unnecessary rows
df_user = df.USER.values.tolist()
rating_user = rating.USER.values.tolist()
todrop_user = list(set(df_user) - set(rating_user))
for name in tqdm(todrop_user) :
    index = df.loc[df.USER == name].index
    df.drop(index, inplace=True)

df.reset_index(drop=True, inplace=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4912/4912 [00:05<00:00, 835.97it/s]


In [72]:
# prepping input for prediction
df.TEXT=df.TEXT.astype(str)
X = tokenizer.texts_to_sequences(df['TEXT'].values)
X = pad_sequences(X, 48)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5863/5863 [00:02<00:00, 1986.31it/s]


In [73]:
# predicting sentiment
pred1 = []
temp = model.predict(X)
polarity = [round(value[1], 3) for value in temp]
for value in polarity :
    pred1.append(0 if value<0.5 else 1)

In [ ]:
# fetching user rating
username = df.USER.values.tolist()
user_rating = []
for x in range(len(username)) :
    user_rating.append(int(rating.loc[rating['USER'] == username[x]]['RATING']))

In [ ]:
# saving output
final = pd.DataFrame(
    list(zip(
        df.USER.values.tolist(),
        user_rating,
        df.TEXT.values.tolist(),
        df.ORIGINAL.values.tolist(),
        df.SENTIMENT.values.tolist(),
        polarity,
        pred1)),
    columns = [ 'USER', 'RATING', 'TEXT', 'ORIGINAL', 'SENTIMENT', 'OUTPUT', 'PRED1' ]
)
final.to_csv("db/09-phase-1-prediction.csv", index=False)

In [ ]:
cm = confusion_matrix(final.SENTIMENT.values.tolist(), final.PRED1.values.tolist())
acc = accuracy_score(final.SENTIMENT.values.tolist(), final.PRED1.values.tolist())
print("Printing Results:")
print(f"Accuracy: {acc}")